# Instituto Tecnologico de Costa Rica
# Ingenieria en Computacion
# Inteligencia artificial 
--- 
## Tarea Programada 3 
## Profesora : Maria Auxialiadora Mora
---
## Estudiantes: 
## Kenneth Corrales Lizano 2017151342
## Fabian Vives Castro 2018
---

In [2]:
#Bibliotecas
import pandas as np
print("Hello")

#Bibliotecas Ejemplo 1 

#Bibliotecas Ejemplo 2

ModuleNotFoundError: No module named 'pandas'

&nbsp;

# Ejercicio 1

### Describa el problema y el objetivo del ejercicio

### Describa los datos utilizados en el ejercicio

### Cargue, normalice y explore los datos


&nbsp;
# Ejercicio 2